# The Battle of Neighborhoods

## Introduction/Business Problem

Fiona comes from a family of chefs. Since moving to the Toronto, she is looking for opportunities to open her own Chinese restaurant to carry on the family tradition. However, she cannot decide the exact location for her business. She knows that she wanted to open the restaurants in the Scarborough area, as there are a lot of Asian immigrants in that borough. There have already been many Chinese restaurants in the area but she is not afraid of the competition. In fact, she intends to use the traffic to her benefit. Her cuisine is unique and she is proud of her skills. She is confident that her cooking will stand out once the business is established. As a result, she intends to place her restaurant in a place with the most dense profile of Chinese restaurants. 

## Data

In [16]:
# Download all the dependencies
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


The Wikipedia will provide a table of postal codes that correlates to the name of the borough and the neighborhood. The table covers the Toronto area and its vicinity. 
Sometimes more than one neighborhood can exist in one postal code area. In those cases, the neighborhoods will be combined into one row with the neighborhoods separated with a comma. If a cell has a borough but not an assigned neighborhood, then the neighborhood will be the same as the borough. 

In [138]:
# Read the tables
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = dfs[0]

# Remove rows with Borough as "Not assigned"
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)

# More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, 
# you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows 
# will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
df1=df.groupby("Postcode").agg(lambda x:','.join(set(x)))

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
# So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
df1.loc[df1['Neighbourhood']=="Not assigned",'Neighbourhood']=df1.loc[df1['Neighbourhood']=="Not assigned",'Borough']

Latitute and longitude data are retrieved online and merged with the table from Wikipedia. The resulting table will contain postal code, borough, neighborhood, latitude, and longitude. The final table will be restricted to only the Scarborough area by selecting boroughs that has the name containing the word "Scarborough". Any postal codes without a specified borough will be excluded.

In [139]:
# Get the latitude and the longitude coordinates for each postal code
latlong = pd.read_csv('https://cocl.us/Geospatial_data')

# Merge latlong and df1
toronto_data = pd.merge(df1, latlong, left_on='Postcode', right_on='Postal Code')

# Select only boroughs that contain the word Toronto
toronto_data = toronto_data[toronto_data['Borough'].str.contains('Scarborough')]
toronto_data.set_index("Postal Code", inplace = True) 
toronto_data.reset_index(inplace = True)
toronto_data

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Ionview,East Birchmount Park,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile,Clairlea,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [148]:
address = 'Scarborough, Ontario'

geolocator = Nominatim(user_agent="ontario_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

# create map
map_Scarborough = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Scarborough)  
    
map_Scarborough

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


The Foursquare location data will be used to help Fiona with her research. The Foursquare location data will provide venue information within a defined radius from Congress. The data will also be used to assess neighborhood characteristics to help Fiona decide which neighborhood should be the home of her restaurant.

In [19]:
# Define Foursquare Credentials and Version
CLIENT_ID = '322IHQFT2MQLFON0WVKHXNIZASFDGJTLERLIMD405ZPONBRK' # your Foursquare ID
CLIENT_SECRET = 'NDBGQEFRKHBAXCGWQUMSIVULCP20YA2YW1R5KY2CW40XNU4H' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 322IHQFT2MQLFON0WVKHXNIZASFDGJTLERLIMD405ZPONBRK
CLIENT_SECRET:NDBGQEFRKHBAXCGWQUMSIVULCP20YA2YW1R5KY2CW40XNU4H


## Methodology 

In [149]:
LIMIT = 200 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

## Results

### Explore the area

First, we'll explore the Scarborough area and look for potential neighborhoods for Fiona to open her restaurant. Candidates will be identified if one of the top five common venues in the neighborhood is Chinese restaurant. We also noticed in the process that Upper Rouge has nothing within the 500 meter radius of its center. Therefore Upper Rouge is removed from the data table. 

In [140]:
# create a function to repeat the same process to all the neighborhoods in Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [141]:
# run the above function on each neighborhood and create a new dataframe called toronto_venues
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Malvern,Rouge
Port Union,Rouge Hill,Highland Creek
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
Ionview,East Birchmount Park,Kennedy Park
Golden Mile,Clairlea,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Scarborough Town Centre,Wexford Heights,Dorset Park
Maryvale,Wexford
Agincourt
Tam O'Shanter,Sullivan,Clarks Corners
Steeles East,L'Amoreaux East,Milliken,Agincourt North
L'Amoreaux West
Upper Rouge


#### Analyze Each Neighborhood

In [142]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()

In [143]:
# write a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [144]:
# create the new dataframe and display the top 10 venues for each neighborhood
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,Clothing Store,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge
1,"Birch Cliff,Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant
2,Cedarbrae,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Gas Station
3,"Cliffcrest,Cliffside,Scarborough Village West",American Restaurant,Skating Rink,Motel,Coffee Shop,Golf Course
4,"Golden Mile,Clairlea,Oakridge",Bakery,Bus Line,Park,Soccer Field,Intersection
5,"Guildwood,Morningside,West Hill",Electronics Store,Intersection,Medical Center,Pizza Place,Rental Car Location
6,"Ionview,East Birchmount Park,Kennedy Park",Discount Store,Hobby Shop,Coffee Shop,Bus Station,Department Store
7,L'Amoreaux West,Chinese Restaurant,Fast Food Restaurant,Burger Joint,Sandwich Place,Grocery Store
8,"Malvern,Rouge",Fast Food Restaurant,Vietnamese Restaurant,Coffee Shop,Grocery Store,Golf Course
9,"Maryvale,Wexford",Bakery,Smoke Shop,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant


In [145]:
toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353,Fast Food Restaurant,Vietnamese Restaurant,Coffee Shop,Grocery Store,Golf Course
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497,Golf Course,Bar,Vietnamese Restaurant,Coffee Shop,Grocery Store
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Electronics Store,Intersection,Medical Center,Pizza Place,Rental Car Location
3,M1G,Scarborough,Woburn,43.770992,-79.216917,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Golf Course,General Entertainment
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Gas Station
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,Playground,Convenience Store,Vietnamese Restaurant,Clothing Store,Golf Course
6,M1K,Scarborough,"Ionview,East Birchmount Park,Kennedy Park",43.727929,-79.262029,Discount Store,Hobby Shop,Coffee Shop,Bus Station,Department Store
7,M1L,Scarborough,"Golden Mile,Clairlea,Oakridge",43.711112,-79.284577,Bakery,Bus Line,Park,Soccer Field,Intersection
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476,American Restaurant,Skating Rink,Motel,Coffee Shop,Golf Course
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant


The result shows that 3 postal code areas fit our selection criteria, namely M1P, M1V nad M1W. Additionally, the M1X area appears to not having any business in the area.

### Search for the specific postal code areas 

In [132]:
latitude_p = 43.757410
longitude_p = -79.273304

latitude_v = 43.815252
longitude_v = -79.284577

latitude_w = 43.799525
longitude_w = -79.318389

search_query = 'Chinese restaurant'
radius = 1000

In [133]:
# Define the corresponding URL
url_p = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_p, longitude_p, VERSION, search_query, radius, LIMIT)

# Send the GET Request and examine the results
results_p = requests.get(url_p).json()

# assign relevant part of JSON to venues
venues_p = results_p['response']['venues']

# tranform venues into a dataframe
dataframe_p = json_normalize(venues_p)

# keep only columns that include venue name, and anything that is associated with location
filtered_columns_p = ['name', 'categories'] + [col for col in dataframe_p.columns if col.startswith('location.')] + ['id']
dataframe_filtered_p = dataframe_p.loc[:, filtered_columns_p]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venues_p.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_p['categories'] = dataframe_filtered_p.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_p.columns = [column.split('.')[-1] for column in dataframe_filtered_p.columns]

dataframe_filtered_p

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Karaikudi Chettinad South Indian Restaurant,Indian Restaurant,1225 Kennedy Rd,CA,Toronto,Canada,at Forbes (Between Lawrence and Ellesmere),283,[1225 Kennedy Rd (at Forbes (Between Lawrence ...,"[{'label': 'display', 'lat': 43.75604153945313...",43.756042,-79.276276,M1P 4Y1,ON,4bf96c435317a593a23a017f
1,Lucky Chinese (Haka) Restaurant,Chinese Restaurant,2650 Lawrence Ave East,CA,Toronto,Canada,at Midland Ave,846,"[2650 Lawrence Ave East (at Midland Ave), Toro...","[{'label': 'display', 'lat': 43.75246481901907...",43.752465,-79.265311,NaN,ON,4bafb9cbf964a520691b3ce3
2,Kim Kim restaurant,Chinese Restaurant,1188 Kennedy Rd,CA,Scarborough,Canada,NaN,478,"[1188 Kennedy Rd, Scarborough ON M1P 2L1, Canada]","[{'label': 'display', 'lat': 43.75383283792732...",43.753833,-79.276611,M1P 2L1,ON,4b6475aef964a520eab42ae3
3,J & Sophie's Place Restaurant,American Restaurant,2623 Lawrence Avenue East,CA,Toronto,Canada,Midland Avenue,973,"[2623 Lawrence Avenue East (Midland Avenue), T...","[{'label': 'display', 'lat': 43.75132259136885...",43.751323,-79.264609,M1P 2S2,ON,4c3b98a062f1c92887c88f53
4,Alan's Filipino Restaurant,Filipino Restaurant,2655 Lawrence Ave. E,CA,Scarborough,Canada,NaN,1104,"[2655 Lawrence Ave. E, Scarborough ON M1P 2S2,...","[{'label': 'display', 'lat': 43.75116420458875...",43.751164,-79.262624,M1P 2S2,ON,56ffe9d4498e8aa2a2123c3d
5,El-Sabil Restaurant,Middle Eastern Restaurant,2680 Lawrence Ave. E,CA,Toronto,Canada,at Brockley Dr.,1009,"[2680 Lawrence Ave. E (at Brockley Dr.), Toron...","[{'label': 'display', 'lat': 43.75240673096773...",43.752407,-79.262830,M1P 4Y4,ON,4eee25244901dff3081ac823


In [134]:
# Define the corresponding URL
url_v = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_v, longitude_v, VERSION, search_query, radius, LIMIT)

# Send the GET Request and examine the results
results_v = requests.get(url_v).json()

# assign relevant part of JSON to venues
venues_v = results_v['response']['venues']

# tranform venues into a dataframe
dataframe_v = json_normalize(venues_v)

# keep only columns that include venue name, and anything that is associated with location
filtered_columns_v = ['name', 'categories'] + [col for col in dataframe_v.columns if col.startswith('location.')] + ['id']
dataframe_filtered_v = dataframe_v.loc[:, filtered_columns_v]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue_v.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_v['categories'] = dataframe_filtered_v.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_v.columns = [column.split('.')[-1] for column in dataframe_filtered_v.columns]

dataframe_filtered_v

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Dong Kee Chinese Restaurant,Chinese Restaurant,3838 Midland Ave.,CA,Scarborough,Canada,Midland + McNicoll,932,"[3838 Midland Ave. (Midland + McNicoll), Scarb...","[{'label': 'display', 'lat': 43.81955848665932...",43.819558,-79.294536,M1V 5K5,ON,4b5cd684f964a5208e4629e3
1,Chinese Bible International 加拿大漢語聖經協會,Bookstore,"3833 Midland Ave, Unit 13",CA,Scarborough,Canada,In Evergold Centre 鴻運中心,578,"[3833 Midland Ave, Unit 13 (In Evergold Centre...","[{'label': 'display', 'lat': 43.81382865109089...",43.813829,-79.291506,M1V 5L6,ON,50edba77e4b071057ef525fd
2,Chinese-Canadian Community Centre,Community Center,"3280 Midland Avenue, Unit 22",CA,Scarbrough,Canada,NaN,1013,"[3280 Midland Avenue, Unit 22, Scarbrough ON M...","[{'label': 'display', 'lat': 43.8071440346539,...",43.807144,-79.290308,M1V 4W9,ON,53493e0e498e89d8ab2333f2
3,Fok Yuan Seafood Restaurant,Chinese Restaurant,3833 Midland Avenue,CA,Scarborough,Canada,McNicoll Avenue,572,"[3833 Midland Avenue (McNicoll Avenue), Scarbo...","[{'label': 'display', 'lat': 43.81414800090627...",43.814148,-79.291543,NaN,ON,4c12709c7f7f2d7fc64add68
4,Lotus Pond Vegetarian Restaurant 蓮花素食,Vegetarian / Vegan Restaurant,3838 Midland Ave.,CA,Scarborough,Canada,btwn Passmore & McNicoll Ave.,934,[3838 Midland Ave. (btwn Passmore & McNicoll A...,"[{'label': 'display', 'lat': 43.81942073443454...",43.819421,-79.294682,M1K 5V5,ON,4b78354df964a52070bc2ee3
5,Scarborough Chinese Alliance Church 仕嘉堡華人宣道會,Church,139 Silver Star Blvd.,CA,Scarborough,Canada,NaN,985,"[139 Silver Star Blvd., Scarborough ON M1V 4V8...","[{'label': 'display', 'lat': 43.80824614802603...",43.808246,-79.292083,M1V 4V8,ON,502a8cf7e4b0f3957219cee7
6,Yee Hong Chinese Evergreen Manor,Building,2319 Mcnicoll,CA,Toronto,Canada,NaN,447,"[2319 Mcnicoll, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.8141508015361,...",43.814151,-79.289941,NaN,ON,4c4225b9af052d7ffc837e79
7,Toronto Chinese Seventh-day Adventist Church,Church,3988 Midland Avenue,CA,Toronto,Canada,NaN,1056,"[3988 Midland Avenue, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.82068336206049...",43.820683,-79.295362,NaN,ON,502fc9bde4b0a20c89126ed7
8,Kitchen Treasure Restaurant Supplies,Furniture / Home Store,3833 Midland Ave. Unit 1,CA,Scarborough,Canada,McNicoll Ave.,569,"[3833 Midland Ave. Unit 1 (McNicoll Ave.), Sca...","[{'label': 'display', 'lat': 43.81400284256661...",43.814003,-79.291447,M1V 5L6,ON,5323cb3a498e24d13677d7ad
9,Fortune Restaurant,Chinese Restaurant,"3833 Midland Ave., Unit 10",CA,Toronto Division,Canada,at McNicoll Ave.,611,"[3833 Midland Ave., Unit 10 (at McNicoll Ave.)...","[{'label': 'display', 'lat': 43.81372159969622...",43.813722,-79.291892,M1V 5L6,ON,4c336f3d7cc0c9b61ccaf19a


In [135]:
# Define the corresponding URL
url_w = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_w, longitude_w, VERSION, search_query, radius, LIMIT)

# Send the GET Request and examine the results
results_w = requests.get(url_w).json()

# assign relevant part of JSON to venues
venues_w = results['response']['venues']

# tranform venues into a dataframe
dataframe_w = json_normalize(venues_w)

# keep only columns that include venue name, and anything that is associated with location
filtered_columns_w = ['name', 'categories'] + [col for col in dataframe_w.columns if col.startswith('location.')] + ['id']
dataframe_filtered_w = dataframe_w.loc[:, filtered_columns_w]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue_w.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_w['categories'] = dataframe_filtered_w.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_w.columns = [column.split('.')[-1] for column in dataframe_filtered_w.columns]

dataframe_filtered_w

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Golden Ray Chinese Restaurant,Cantonese Restaurant,NaN,CA,NaN,Canada,NaN,1246,[Canada],"[{'label': 'display', 'lat': 43.79809, 'lng': ...",43.798090,-79.333780,NaN,NaN,NaN,56c7b56bcd1080d7a204a1cc
1,Best Chinese Restaurant,Chinese Restaurant,NaN,CA,Toronto,Canada,NaN,1827,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.810921, 'lng':...",43.810921,-79.302008,NaN,NaN,ON,4cfa6dc7c6cca35df42c8632
2,Mr Congee Chinese Cuisine 龍粥記,Chinese Restaurant,2900 Warden Ave,CA,Toronto,Canada,at Finch Ave. E,72,"[2900 Warden Ave (at Finch Ave. E), Toronto ON...","[{'label': 'display', 'lat': 43.79887879258761...",43.798879,-79.318335,L'Amoreaux,M1W 2S8,ON,4df28ea0e4cda09e6da0129a
3,Jasmine Chinese Food,Chinese Restaurant,2900 Warden Avenue,CA,Scarborough,Canada,At Finch Avenue East,22,"[2900 Warden Avenue (At Finch Avenue East), Sc...","[{'label': 'display', 'lat': 43.79971978534943...",43.799720,-79.318469,NaN,NaN,ON,4ddef195c65bb5e319c6e124
4,Phoenix Restaurant 金鳳餐廳,Chinese Restaurant,2900 Warden Ave,CA,Scarborough,Canada,at Finch Ave E,147,"[2900 Warden Ave (at Finch Ave E), Scarborough...","[{'label': 'display', 'lat': 43.79819775434163...",43.798198,-79.318432,NaN,M1W 2S8,ON,5542be72498e851f264ce89a
5,Fisherman Lobster King,Chinese Restaurant,2900 Warden Ave.,CA,Toronto,Canada,Finch and Warden,87,"[2900 Warden Ave. (Finch and Warden), Toronto ...","[{'label': 'display', 'lat': 43.798744, 'lng':...",43.798744,-79.318230,NaN,M1W 2S8,ON,5cdf4ba98b98fd00393eab76
6,Landmark Seafood Cuisine 富臨門海鮮酒家,Chinese Restaurant,3601 Victoria Park Ave.,CA,Toronto,Canada,at McNicoll Ave.,1490,"[3601 Victoria Park Ave. (at McNicoll Ave.), T...","[{'label': 'display', 'lat': 43.80402612369180...",43.804026,-79.335857,NaN,M1W 3Y3,ON,4ef0cc69991107885c190eb3
7,Chinese Gospel Church Scarborough,Church,NaN,CA,NaN,Canada,NaN,1570,[Canada],"[{'label': 'display', 'lat': 43.79098598346052...",43.790986,-79.302830,NaN,NaN,NaN,4c5c9cefd25320a1a3c2c37a
8,Home Feel Chinese Seafood Restaurant,Chinese Restaurant,NaN,CA,Toronto,Canada,NaN,1233,"[Toronto ON M1W 2S6, Canada]","[{'label': 'display', 'lat': 43.798465, 'lng':...",43.798465,-79.333677,NaN,M1W 2S6,ON,5d30ecb7898bc70008b2c7f2
9,Yang Ting Restaurant,Chinese Restaurant,"2900 Warden Ave, unit # 107",CA,Toronto,Canada,Warden and Finch Ave E,198,"[2900 Warden Ave, unit # 107 (Warden and Finch...","[{'label': 'display', 'lat': 43.797768, 'lng':...",43.797768,-79.317980,NaN,M1W 2S8,ON,5c3fa780029a5500397542e4


Detailed searches show that the M1P, M1V, and M1W area actually have 2, 7, and 12 Chinese restaurants. 

## Discussion

Our exploration of the Scarborough borough revealed three potential postal code areas for Fiona to open her restaurant in. However, we recognize that on websites such as Foursquare, one doesn't necessarily only receive results related to the search keyword. There are results returned that are deemed appropriate by the algorithm but are irrelevant from human perspective. As a result, we also performed detailed search of Chinese restaurant on the three area candidate. The result shows that if Fiona wants to benefit from the traffic generated by the currently existing restaurants, then she should find a place in the M1W postal code area. 

## Conclusion

According to the study, Fiona should start looking for venues in the M1W postal code area to open her restaurant.